In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Define all needed functions for CART

In [2]:
# accuracy function (number of samples where predictions and actual value matched)
def accuracy(actual, predicted):
    return sum(actual==predicted)/len(predicted)

In [3]:
# Split a dataset depends on attribute/value
def test_split(index, value, dataset):
	left, right = [], []
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [4]:
# Calculate gini index
def gini_index(groups, classes):
	n_instances = sum([len(group) for group in groups])
	gini = 0
	for group in groups:
		size = len(group)
		# zero check
		if size == 0:
			continue
		score = 0
		for class_val in classes:
			p = (1/size)*([row[-1] for row in group].count(class_val))
			score += p * p
        # final formula
		gini += (1 - score) * (size / n_instances)
	return gini

In [5]:
# search for the best split
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
    # initial values
	b_index, b_value, b_score, b_groups = np.inf, np.inf, np.inf, None
    # for all attributes in a dataset
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    # returns dict with the best split
	return {'index':b_index, 'value':b_value, 'groups':b_groups}


In [6]:
# Create a terminal node value (there`s no other nodes below)
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

In [7]:
# Create splits 
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [8]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [9]:
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [10]:
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)    

## Generate data (same algorithm as in logistic regression)

In [11]:
num_samples,bias = 100,1.1
# make a simple unit circle 
theta = np.linspace(0, 2*np.pi, num_samples)
# make 2 circles with different labels
r1,r2 = np.random.rand((num_samples)),np.random.rand((num_samples))
x1, y1 = (r1 * np.cos(theta)).reshape(-1, 1), (r1 * np.sin(theta)).reshape(-1, 1)
x2, y2 = (r2 * np.cos(theta) + bias).reshape(-1, 1), (r2 * np.sin(theta) + bias).reshape(-1, 1)

s1 = np.hstack([x1,y1, np.ones_like(x1)])
s2 = np.hstack([x2,y2, np.zeros_like(x2)])
data = np.vstack([s1,s2])
data= np.hstack([np.ones([data.shape[0],1]),data])

### Split dataset (train/test)

In [12]:
np.random.shuffle(data)
# 70/30
split_ind = int(np.floor((num_samples*7)/10))
split_ind
train,test = data[split_ind:],data[:split_ind]
(x_train,y_train) = (train[:,:data.shape[1]-1],train[:,-1])
(x_test,y_test) = (test[:,:data.shape[1]-1],test[:,-1])

### Fit model and make predictions on the test split

In [13]:
# params
max_depth, min_size = 5,10
predictions = decision_tree(train, test, max_depth, min_size)
accuracy(predictions,y_test)

0.9428571428571428